In [1]:
from itertools import chain

import pandas as pd

from ddeserts.annotate import RACES
from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import add_geo_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data
from ddeserts.load import load_charter_cities

In [2]:
# load the data
pre_filter = lambda line: ('California' in line and 'CDP' not in line)

df = load_cvap_data('Place', pre_filter=pre_filter)
len(df)

485

In [3]:
# add geo columns and post-filter
add_geo_columns(df)
df = df[(df['state'] == 'California') & df['geotype'].isin(['city', 'town'])]
len(df)

482

In [4]:
# add charter city data
charter_cities = load_charter_cities()
df['geosubtype'] = 'general law'
df.loc[df['name'].isin(charter_cities), 'geosubtype'] = 'charter'

In [5]:
# add stat columns and sort
add_all_stat_columns(df)
df = with_columns_sorted(df)

In [6]:
# sort by common name of city (Ventura, not San Buenaventura)
df.sort_values(by=['name'], inplace=True)

In [7]:
df

geoid                        geoname   geosubtype  \
table line                                                                  
Place 26560  1600000US0600296      Adelanto city, California      charter   
      26586  1600000US0600394  Agoura Hills city, California  general law   
      26651  1600000US0600562       Alameda city, California      charter   
      26677  1600000US0600674        Albany city, California      charter   
      26716  1600000US0600884      Alhambra city, California      charter   
...                       ...                            ...          ...   
      47386  1600000US0686930    Yountville city, California  general law   
      47399  1600000US0686944         Yreka city, California  general law   
      47412  1600000US0686972     Yuba City city, California  general law   
      47425  1600000US0687042       Yucaipa city, California  general law   
      47438  1600000US0687056  Yucca Valley town, California  general law   

            geotype          name       state  adu_est  adu_moe  cit_est  \
table line                                                                 
Place 26560    city      Adelanto  California    24830      882    33815   
      26586    city  Agoura Hills  California    15880      602    19480   
      26651    city       Alameda  California    61780      691    72305   
      26677    city        Albany  California    14790      329    17045   
      26716    city      Alhambra  California    68630      717    70150   
...             ...           ...         ...      ...      ...      ...   
      47386    city    Yountville  California     3065       72     3275   
      47399    city         Yreka  California     5705      304     7750   
      47412    city     Yuba City  California    51885      473    61875   
      47425    city       Yucaipa  California    40510      954    51940   
      47438    city  Yucca Valley  California    16650      479    21070   

             cit_moe  ...  prop_his_adu_dvap_est  prop_his_adu_dvap_moe  \
table line            ...                                                 
Place 26560      572  ...               0.201182               0.074798   
      26586      323  ...               0.071240               0.366285   
      26651      727  ...               0.143922               0.122714   
      26677      405  ...               0.194888               0.203390   
      26716     1081  ...               0.124564               0.056432   
...              ...  ...                    ...                    ...   
      47386       75  ...               0.047619               0.569416   
      47399       56  ...               0.059880               0.322946   
      47412      793  ...               0.303074               0.062663   
      47425      474  ...               0.131088               0.108338   
      47438      237  ...               0.094388               0.202449   

             prop_ind_adu_dvap_est  prop_ind_adu_dvap_moe  \
table line                                                  
Place 26560                    0.0               1.000000   
      26586                    NaN                    NaN   
      26651                    0.0               0.572756   
      26677                    0.0               1.000000   
      26716                    0.0               0.896672   
...                            ...                    ...   
      47386                    0.0               1.000000   
      47399                    0.0               0.557604   
      47412                    0.0               0.325137   
      47425                    0.0               1.000000   
      47438                    0.0               1.000000   

             prop_pac_adu_dvap_est  prop_pac_adu_dvap_moe  \
table line                                                  
Place 26560               0.357143               0.642857   
      26586                    NaN                    NaN   
      26651        

In [8]:
df.to_csv('output/ca-cities.csv')

In [9]:
# general disenfranchisement data
cols = (
    ['name', 'geosubtype'] +
    list(chain(*(
        [f'{pop}_est', f'{pop}_moe'] 
        for pop in ('tot', 'adu', 'dvap', 'prop_adu_dvap')
    ))) +
    list(chain(*(
        [f'{r}_dvap_est', f'{r}_dvap_moe', 
         f'prop_{r}_adu_dvap_est', f'prop_{r}_adu_dvap_moe']
        for r in ('wht', 'his', 'asn', 'blk', 'ind', 'pac', 'tmr')
    )))
)
dg = df[cols]
dg.to_csv('output/ca-cities-dvap.csv')

In [10]:
# racial disparity data
cols = (
    ['name', 'geosubtype'] + 
    list(chain(*(
        [f'{pop}_est', f'{pop}_moe'] for pop in ('tot', 'adu', 'cvap')
    ))) +
    list(chain(*(
        [f'prop_adu_{r}_est', f'prop_cvap_{r}_est', 
         f'racial_disp_{r}_est', f'racial_disp_{r}_moe']
        for r in ('wht', 'his', 'asn', 'blk', 'ind', 'pac', 'tmr')
    )))
)
dg = df[cols]
dg.to_csv('output/ca-cities-racial-disp.csv')

In [11]:
dg = df[['name', 'geosubtype', 'racial_disp_his_est', 'racial_disp_his_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_his_est'])[:30]

name geosubtype  racial_disp_his_est  \
table line                                                        
Place 42706             Salinas    charter            -0.118589   
      43239          San Rafael    charter            -0.113010   
      43369         Santa Maria    charter            -0.110375   
      42901           Sand City    charter            -0.108136   
      35738           King City    charter            -0.100792   
      35166           Inglewood    charter            -0.097765   
      42056            Richmond    charter            -0.093663   
      30499             Compton    charter            -0.090163   
      46424         Watsonville    charter            -0.085065   
      27002             Anaheim    charter            -0.083366   
      43291       Santa Barbara    charter            -0.079453   
      43421          Santa Rosa    charter            -0.077573   
      39352                Napa    charter            -0.075754   
      41406         Porterville    charter            -0.075488   
      37428         Los Angeles    charter            -0.075364   
      43278           Santa Ana    charter            -0.073264   
      43733             Shafter    charter            -0.071422   
      31383  Desert Hot Springs    charter            -0.070352   
      39872             Oakland    charter            -0.069681   
      35153            Industry    charter            -0.067130   
      38338              Merced    charter            -0.063374   
      36999             Lindsay    charter            -0.060248   
      41952        Redwood City    charter            -0.059288   
      37337          Long Beach    charter            -0.059245   
      46216               Vista    charter            -0.059123   
      41419        Port Hueneme    charter            -0.058744   
      42836      San Bernardino    charter            -0.057359   
      45605              Tulare    charter            -0.056936   
      43135          San Marcos    charter            -0.055992   
      40431            Palmdale    charter            -0.055707   

             racial_disp_his_moe  
table line                        
Place 42706             0.026648  
      43239             0.023344  
      43369             0.036510  
      42901             0.225121  
      35738             0.192112  
      35166             0.030867  
      42056             0.028870  
      30499             0.037342  
      46424             0.061124  
      27002             0.017250  
      43291             0.027224  
      43421             0.016600  
      39352             0.019256  
      41406             0.055019  
      37428             0.004158  
      43278             0.023167  
      43733             0.092877  
      31383             0.071939  
      39872             0.010227  
      35153             0.374987  
      38338             0.033340  
      36999             0.162315  
      41952             0.029604  
      37337             0.013210  
      46216             0.031728  
      41419             0.066123  
      42836             0.025354  
      45605             0.045862  
      43135             0.036746  
      40431             0.032284

In [12]:
dg = df[['name', 'geosubtype', 'racial_disp_asn_est', 'racial_disp_asn_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_asn_est'])[:30]

name geosubtype  racial_disp_asn_est  \
table line                                                   
Place 44825      Sunnyvale    charter            -0.141770   
      43304    Santa Clara    charter            -0.135994   
      26677         Albany    charter            -0.081785   
      35244         Irvine    charter            -0.076732   
      39183  Mountain View    charter            -0.076523   
      43252      San Ramon    charter            -0.069550   
      27158        Arcadia    charter            -0.061197   
      45449       Torrance    charter            -0.060456   
      31032        Cypress    charter            -0.050265   
      27938       Berkeley    charter            -0.049973   
      33047         Folsom    charter            -0.047856   
      26716       Alhambra    charter            -0.047124   
      40470      Palo Alto    charter            -0.044707   
      30967    Culver City    charter            -0.041592   
      43174      San Mateo    charter            -0.038360   
      43044       San Jose    charter            -0.031869   
      35738      King City    charter            -0.030676   
      41952   Redwood City    charter            -0.030557   
      28757        Burbank    charter            -0.028067   
      43330     Santa Cruz    charter            -0.026460   
      29771       Cerritos    charter            -0.026274   
      40652       Pasadena    charter            -0.025867   
      43083    San Leandro    charter            -0.024717   
      26651        Alameda    charter            -0.024535   
      42914      San Diego    charter            -0.024467   
      42966  San Francisco    charter            -0.021586   
      28731     Buena Park    charter            -0.020857   
      45150    Temple City    charter            -0.020606   
      38871       Monterey    charter            -0.019602   
      43135     San Marcos    charter            -0.017775   

             racial_disp_asn_moe  
table line                        
Place 44825             0.026585  
      43304             0.021735  
      26677             0.048350  
      35244             0.021082  
      39183             0.027209  
      43252             0.032210  
      27158             0.049032  
      45449             0.021651  
      31032             0.034343  
      27938             0.019378  
      33047             0.019951  
      26716             0.029939  
      40470             0.029488  
      30967             0.029933  
      43174             0.022696  
      43044             0.006842  
      35738             0.037217  
      41952             0.021341  
      28757             0.016532  
      43330             0.011848  
      29771             0.040653  
      40652             0.016416  
      43083             0.030951  
      26651             0.026179  
      42914             0.004329  
      42966             0.004694  
      28731             0.039710  
      45150             0.055768  
      38871             0.019166  
      43135             0.022126

In [13]:
dg = df[['name', 'geosubtype', 'racial_disp_wht_est', 'racial_disp_wht_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_wht_est'], ascending=False)[:30]

name geosubtype  racial_disp_wht_est  \
table line                                                        
Place 35738           King City    charter             0.120704   
      44825           Sunnyvale    charter             0.118399   
      43239          San Rafael    charter             0.114817   
      39183       Mountain View    charter             0.096644   
      43304         Santa Clara    charter             0.095780   
      42901           Sand City    charter             0.091525   
      43369         Santa Maria    charter             0.088179   
      43421          Santa Rosa    charter             0.083254   
      42706             Salinas    charter             0.083180   
      35153            Industry    charter             0.081019   
      43291       Santa Barbara    charter             0.079062   
      41952        Redwood City    charter             0.078513   
      39352                Napa    charter             0.074207   
      46424         Watsonville    charter             0.069805   
      26677              Albany    charter             0.069378   
      43174           San Mateo    charter             0.068660   
      43135          San Marcos    charter             0.065841   
      41406         Porterville    charter             0.064617   
      27002             Anaheim    charter             0.063094   
      43733             Shafter    charter             0.060016   
      35244              Irvine    charter             0.058262   
      37428         Los Angeles    charter             0.057774   
      36999             Lindsay    charter             0.056711   
      46216               Vista    charter             0.056030   
      31383  Desert Hot Springs    charter             0.054539   
      38338              Merced    charter             0.053553   
      43252           San Ramon    charter             0.053219   
      45605              Tulare    charter             0.053214   
      40847            Petaluma    charter             0.052675   
      40652            Pasadena    charter             0.052648   

             racial_disp_wht_moe  
table line                        
Place 35738             0.085067  
      44825             0.022254  
      43239             0.042717  
      39183             0.031462  
      43304             0.024875  
      42901             0.315872  
      43369             0.028587  
      43421             0.021193  
      42706             0.017692  
      35153             0.340572  
      43291             0.036658  
      41952             0.034763  
      39352             0.022847  
      46424             0.045957  
      26677             0.056317  
      43174             0.031151  
      43135             0.040074  
      41406             0.039497  
      27002             0.015075  
      43733             0.044924  
      35244             0.020017  
      37428             0.003632  
      36999             0.072515  
      46216             0.032822  
      31383             0.053888  
      38338             0.027949  
      43252             0.033974  
      45605             0.038249  
      40847             0.036279  
      40652             0.022641

In [14]:
dg = df[df['name'] == 'King City']
dg[['prop_adu_his_est', 'prop_adu_his_moe', 'prop_dvap_his_est', 'prop_dvap_his_moe', 
    'prop_adu_dvap_est', 'prop_adu_dvap_moe', 'prop_his_adu_dvap_est', 'prop_his_adu_dvap_moe', 
    'adu_est', 'cvap_est', 'his_adu_est', 'his_adu_moe', 'his_cvap_est', 'his_cvap_moe']]

,,prop_adu_his_est,prop_adu_his_moe,prop_dvap_his_est,prop_dvap_his_moe,prop_adu_dvap_est,prop_adu_dvap_moe,prop_his_adu_dvap_est,prop_his_adu_dvap_moe,adu_est,cvap_est,his_adu_est,his_adu_moe,his_cvap_est,his_cvap_moe
table,line,,,,,,,,,,,,,,
Place,35738,0.830805,0.078194,0.911672,0.212565,0.554842,0.070018,0.608848,0.080067,8570,3815,7120,568,2785,525


In [15]:
dg = df[df['name'] == 'King City']
dg[['prop_adu_his_est', 'prop_cvap_his_est', 'racial_disp_his_est', 'racial_disp_his_moe']]

,,prop_adu_his_est,prop_cvap_his_est,racial_disp_his_est,racial_disp_his_moe
table,line,,,,
Place,35738,0.830805,0.730013,-0.100792,0.192112


In [16]:
df[['name', 'prop_adu_dvap_moe']]

name  prop_adu_dvap_moe
table line                                  
Place 26560      Adelanto           0.046244
      26586  Agoura Hills           0.051033
      26651       Alameda           0.017271
      26677        Albany           0.037088
      26716      Alhambra           0.018596
...                   ...                ...
      47386    Yountville           0.038285
      47399         Yreka           0.075246
      47412     Yuba City           0.019660
      47425       Yucaipa           0.034674
      47438  Yucca Valley           0.042815

[482 rows x 2 columns]

In [17]:
len(df[df['tot_est'] >= 50000])

180